In [3]:
import pandas as pd
import hvplot.pandas
import glob
import os

CARGAR TODOS LOS ARCHIVOS Y CONCATENARLOS TODOS

In [4]:
#Hacer una lista con los archivos de la carpeta
archivos = glob.glob("/Users/nacho/Desktop/TFM/Datos/Datos equipos LaLiga J27/*xlsx")
len(archivos)

20

In [5]:
#Cargar los archivos
len(archivos)

lista_de_archivos = []
for f in archivos:
    data = pd.read_excel(f)
    nombre = os.path.basename(f)
    data['carpeta_contenedora'] = nombre
    lista_de_archivos.append(data)

In [6]:
len(lista_de_archivos)

20

In [7]:
#Concatenarlos en un solo data frame
df = pd.concat(lista_de_archivos, ignore_index= True)

In [8]:
df.shape

(1396, 105)

LIMPIAR Y ORDENAR LOS DATOS DEL ARCHIVO CONCATENADO

In [9]:
#Eliminar filas de cada partido, solo nos quedamos con las generales, las estadísticas de lo equipos y las de los rivales contra cada equipo.
df = df[~(df['Team'].str.startswith('2022') | df['Team'].str.startswith('2023'))]

In [10]:
#Crear dos df distientos uno con las estadísticas de lo equipos y otros con las estadísticas de los rivales contra cada equipo.
Opponents_stats = df[df['Team'].str.startswith('Opponents')]
Team_stats = df[~df['Team'].str.startswith('Opponents')]

ANÁLISIS DE MODELO DE JUEGO

1 FASE OFENSIVA FINALIZACIÓN

1.1 Goles / Xg

In [18]:
import hvplot.pandas
import holoviews as hv
import numpy as np
import matplotlib.pyplot as plt
from bokeh.io import export_png

In [12]:
# Ordenar los equipos por número de goles
Team_stats = Team_stats.sort_values('Goals', ascending=True)

# Crear el gráfico
goles_90 = Team_stats.hvplot.barh(x='Team', y='Goals',
                       color='Goals', cmap='RdYlGn', 
                       xlabel='Equipo', ylabel='Goles a favor por 90', 
                       title='Goles por 90 de los equipos de LaLiga')

goles_90

:Bars   [Team]   (Goals)

In [13]:
# Ordenar los equipos por xG.
Team_stats = Team_stats.sort_values('xG', ascending=True)

# Crear el gráfico
xg_90 = Team_stats.hvplot.barh(x='Team', y='xG',
                       color='xG', cmap='RdYlGn', 
                       xlabel='Equipo', ylabel='xG creado por 90', 
                       title='xG creado por 90 de los equipos de LaLiga\n(Calidad de las acciones)',
                       line_width=[50 if team=='Espanyol' else 1 for team in Team_stats['Team']])

xg_90

:Bars   [Team]   (xG)

In [14]:
# Crear una nueva columna que mida la eficacia goleadora. Para comparar la calidad de las acciones y los goles.
Team_stats['Eficacia goleadora (Goals - xG)'] = round(Team_stats['Goals'] - Team_stats['xG'] , 5)

# Ordenar los equipos por xG.
Team_stats = Team_stats.sort_values('Eficacia goleadora (Goals - xG)', ascending=True)

# Crear el gráfico
eficacia_goleadora = Team_stats.hvplot.barh(x='Team', y='Eficacia goleadora (Goals - xG)',
                       color='Eficacia goleadora (Goals - xG)', cmap='RdYlGn', 
                       xlabel='Equipo', ylabel='Eficacia goleadora (Goles - xG)', 
                       title='Eficacia goleadora por 90 de los equipos de LaLiga',
                       hover_cols=["Team", "Goals", "xG"])

eficacia_goleadora

:Bars   [Team]   (Eficacia goleadora (Goals - xG),Goals,xG)

1.2 Tiros

In [15]:
import holoviews as hv

#Columna de tiros necesarios para hacer gol
Team_stats['Shots per Goal'] = Team_stats['Shots'] / Team_stats['Goals']

# Calcular las medias de las columnas 'Goals' y 'Shots'
goals_mean = Team_stats['Goals'].mean()
shots_mean = Team_stats['Shots'].mean()

# Crear una nueva columna de colores en el DataFrame
Team_stats['color'] = ['blue' if team == 'Espanyol' else 'grey' for team in Team_stats['Team']]

# Crear el gráfico con los colores específicos para cada equipo
plot = Team_stats.hvplot.scatter(x='Goals', y='Shots', by='Team', 
                                 legend='right', height=600, width=900, color='color', 
                                 hover_cols=["Team", "Goals", "Shots", "Shots per Goal"])

#Añadir lineas de media de goles y tiros para ver cuadrantes
shots_mean_line = hv.HLine(shots_mean)
shots_mean_line.opts(
    color='grey', 
    line_dash='dashed', 
    line_width=1,
)

goals_mean_line = hv.VLine(goals_mean)
goals_mean_line.opts(
    color='grey', 
    line_dash='dashed', 
    line_width=1,
)

relacion_tiros_goles = plot * shots_mean_line * goals_mean_line.opts(title='Tiros / Goles por 90')

relacion_tiros_goles

:Overlay
   .NdOverlay.I :NdOverlay   [Team]
      :Scatter   [Goals]   (Shots,color,Team,Shots per Goal)
   .HLine.I     :HLine   [x,y]
   .VLine.I     :VLine   [x,y]

In [16]:
# Ordenar los equipos por % of Shots on target.
Team_stats = Team_stats.sort_values('% of Shots on target', ascending=True)

# Crear el gráfico
tiros_puerta_porcentaje = Team_stats.hvplot.barh(x='Team', y='% of Shots on target',
                       color='% of Shots on target', cmap='RdYlGn', 
                       xlabel='Equipo', ylabel='% tiros a puerta por 90', 
                       title='Porcentaje de tiros a puerta por 90')

tiros_puerta_porcentaje

:Bars   [Team]   (% of Shots on target)

In [17]:
Team_stats['Total shots from inside penalty area'] = Team_stats['Shots'] - Team_stats['Total shots from outside penalty areat']

#Calcular las medias de las columnas de centros y pases profundos.
shots_outside_mean = Team_stats['Total shots from outside penalty areat'].mean()
shots_inside_mean = Team_stats['Total shots from inside penalty area'].mean()

#Crear el gráfico con los colores específicos para cada equipo
plot = Team_stats.hvplot.scatter(x='Total shots from outside penalty areat', y='Total shots from inside penalty area', by='Team', 
                                 legend='right', height=600, width=900, color='color', 
                                 hover_cols=["Total shots from outside penalty areat", "Total shots from inside penalty area"])

#Añadir lineas de media de goles y tiros para ver cuadrantes
shots_outside_mean_line = hv.VLine(shots_outside_mean)
shots_outside_mean_line.opts(
    color='grey', 
    line_dash='dashed', 
    line_width=1,
)

shots_inside_mean_line = hv.HLine(shots_inside_mean)
shots_inside_mean_line.opts(
    color='grey', 
    line_dash='dashed', 
    line_width=1,
)

localizacion_tiros = plot * shots_outside_mean_line * shots_inside_mean_line.opts(title='Localización de tiros')

localizacion_tiros

:Overlay
   .NdOverlay.I :NdOverlay   [Team]
      :Scatter   [Total shots from outside penalty areat]   (Total shots from inside penalty area,color)
   .VLine.I     :VLine   [x,y]
   .HLine.I     :HLine   [x,y]

1.3 Centros

In [19]:
# Ordenar los equipos por centros.
Team_stats = Team_stats.sort_values('Crosses', ascending=True)

# Crear el gráfico
centros = Team_stats.hvplot.barh(x='Team', y='Crosses',
                       color='Crosses', cmap='RdYlGn', 
                       xlabel='Equipo', ylabel='Centros', 
                       title='Total de centros al área por 90')

centros

:Bars   [Team]   (Crosses)

In [20]:
# Ordenar los equipos por % de centros exitosos.
Team_stats = Team_stats.sort_values('% of Crosses accurate', ascending=True)

# Crear el gráfico
centros_exitosos_porcentaje = Team_stats.hvplot.barh(x='Team', y='% of Crosses accurate',
                       color='% of Crosses accurate', cmap='RdYlGn', 
                       xlabel='Equipo', ylabel='Centros exitosos', 
                       title='Porcentajes de centros exitosos al área por 90')

centros_exitosos_porcentaje

:Bars   [Team]   (% of Crosses accurate)

1.4 Acciones último tercio

In [21]:
# Calcular las medias de las columnas de centros y pases profundos.
deep_completed_crosses_mean = Team_stats['Deep completed crosses'].mean()
deep_completed_passes_mean = Team_stats['Deep completed passes'].mean()

# Crear el gráfico con los colores específicos para cada equipo
plot = Team_stats.hvplot.scatter(x='Deep completed crosses', y='Deep completed passes', by='Team', 
                                 legend='right', height=600, width=900, color='color', 
                                 hover_cols=["Deep completed crosses", "Deep completed passes"])

#Añadir lineas de media de goles y tiros para ver cuadrantes
deep_completed_crosses_mean_line = hv.VLine(deep_completed_crosses_mean)
deep_completed_crosses_mean_line.opts(
    color='grey', 
    line_dash='dashed', 
    line_width=1,
)

deep_completed_passes_mean_line = hv.HLine(deep_completed_passes_mean)
deep_completed_passes_mean_line.opts(
    color='grey', 
    line_dash='dashed', 
    line_width=1,
)

acciones_profundidad = plot * deep_completed_crosses_mean_line * deep_completed_passes_mean_line.opts(title='Acciones con profundidad por 90')

acciones_profundidad

:Overlay
   .NdOverlay.I :NdOverlay   [Team]
      :Scatter   [Deep completed crosses]   (Deep completed passes,color)
   .VLine.I     :VLine   [x,y]
   .HLine.I     :HLine   [x,y]

In [22]:
# Ordenar los equipos por entradas al área.
Team_stats = Team_stats.sort_values('Penalty area entries', ascending=True)

# Crear el gráfico
entradas_area = Team_stats.hvplot.barh(x='Team', y='Penalty area entries',
                       color='Penalty area entries', cmap='RdYlGn', 
                       xlabel='Equipo', ylabel='Entradas al área por 90', 
                       title='Entradas al área por 90')

entradas_area

:Bars   [Team]   (Penalty area entries)

In [23]:
#Calcular la nueva columna, Ratio de eficacia
Team_stats['Scoring rate'] = Team_stats['Penalty area entries'] / Team_stats['Goals']

# Ordenar los equipos por Ratio de eficacia.
Team_stats = Team_stats.sort_values('Scoring rate', ascending=False)

# Crear el gráfico
ratio_eficacia = Team_stats.hvplot.barh(x='Team', y='Scoring rate',
                       color='Scoring rate', cmap='RdYlGn_r', 
                       xlabel='Equipo', ylabel='Ratio de eficacia por 90', 
                       title='Ratio de eficacia por 90\n(Entradas al área / Goles)')

ratio_eficacia

:Bars   [Team]   (Scoring rate)

In [24]:
# Calcular el porcentaje de Penalty area runs
Team_stats['% Penalty area runs'] = 100 * Team_stats['Penalty area runs'] / Team_stats['Penalty area entries']
# Calcular el porcentaje de Penalty area crosses
Team_stats['% Penalty area entries crosses'] = 100 * Team_stats['Penalty area entries crosses'] / Team_stats['Penalty area entries']

# Calcular las medias de las columnas de % Penalty area runs y % Penalty area entries crosses.
penalty_area_runs_mean = Team_stats['% Penalty area runs'].mean()
penalty_area_entries_crosses_mean = Team_stats['% Penalty area entries crosses'].mean()

# Crear una nueva columna de colores en el DataFrame
Team_stats['color'] = ['blue' if team == 'Espanyol' else 'grey' for team in Team_stats['Team']]

# Crear el gráfico con los colores específicos para cada equipo
plot = Team_stats.hvplot.scatter(x='% Penalty area runs', y='% Penalty area entries crosses', by='Team', 
                                 legend='right', height=600, width=900, color='color', 
                                 hover_cols=["% Penalty area runs", "% Penalty area entries crosses"])

#Añadir lineas de media de goles y tiros para ver cuadrantes
penalty_area_runs_mean_line = hv.VLine(penalty_area_runs_mean)
penalty_area_runs_mean_line.opts(
    color='grey', 
    line_dash='dashed', 
    line_width=1,
)

penalty_area_entries_crosses_mean_line = hv.HLine(penalty_area_entries_crosses_mean)
penalty_area_entries_crosses_mean_line.opts(
    color='grey', 
    line_dash='dashed', 
    line_width=1,
)

preferencia_entradas_area= plot * penalty_area_runs_mean_line * penalty_area_entries_crosses_mean_line.opts(title='Entradas al área por 90')

preferencia_entradas_area

:Overlay
   .NdOverlay.I :NdOverlay   [Team]
      :Scatter   [% Penalty area runs]   (% Penalty area entries crosses,color)
   .VLine.I     :VLine   [x,y]
   .HLine.I     :HLine   [x,y]

In [25]:
# Ordenar los equipos por toques dentro del área.
Team_stats = Team_stats.sort_values('Touches in penalty area', ascending=True)

# Crear el gráfico
toques_dentro_área = Team_stats.hvplot.barh(x='Team', y='Touches in penalty area',
                       color='Touches in penalty area', cmap='RdYlGn', 
                       xlabel='Equipo', ylabel='Toques dentro del área', 
                       title='Toques dentro del área por 90')

toques_dentro_área

:Bars   [Team]   (Touches in penalty area)

1.5 Duelos aéreos

In [26]:
# Calcular las medias de las columnas de duelos aéreos y duelos aéreos ganados.
aerial_duels_mean = Team_stats['Aerial duels'].mean()
aerial_duels_won_mean = Team_stats['Aerial duels won'].mean()

# Crear una nueva columna de colores en el DataFrame
Team_stats['color'] = ['blue' if team == 'Espanyol' else 'grey' for team in Team_stats['Team']]

# Crear el gráfico con los colores específicos para cada equipo
duelos_areos_visualización = Team_stats.hvplot.scatter(x='Aerial duels', y='Aerial duels won', by='Team', 
                                 legend='right', height=600, width=900, color='color', 
                                 hover_cols=["Aerial duels", "Aerial duels won"])

#Añadir lineas de media de goles y tiros para ver cuadrantes
aerial_duels_mean_line = hv.VLine(aerial_duels_mean)
aerial_duels_mean_line.opts(
    color='grey', 
    line_dash='dashed', 
    line_width=1,
)

aerial_duels_won_mean_line = hv.HLine(aerial_duels_won_mean)
aerial_duels_won_mean_line.opts(
    color='grey', 
    line_dash='dashed', 
    line_width=1,
)

duelos_areos = duelos_areos_visualización * aerial_duels_won_mean_line * aerial_duels_mean_line.opts(title='Duelos aéreos')

duelos_areos

:Overlay
   .NdOverlay.I :NdOverlay   [Team]
      :Scatter   [Aerial duels]   (Aerial duels won,color)
   .HLine.I     :HLine   [x,y]
   .VLine.I     :VLine   [x,y]

In [27]:
#Guardar visualizaciones. Nombre = fase ofesniva(1)_subfase(1,2,3...)_visualización

#1. Goles / xG
hvplot.save(goles_90, '1_1_goles_90.png')
hvplot.save(xg_90, '1_1_xg_90.png')
hvplot.save(eficacia_goleadora, '1_1_eficacia_goleadora.png') #goles - xg

#2. Tiros
hvplot.save(relacion_tiros_goles, '1_2_relacion_tiros_goles.png') #tiros por cada gol
hvplot.save(tiros_puerta_porcentaje, '1_2_tiros_puerta_porcentaje.png')
hvplot.save(localizacion_tiros, '1_2_localizacion_tiros.png') #tiros desde fuera / tiros desde dentro

#1.3 Centros
hvplot.save(centros, '1_3_centros.png')
hvplot.save(centros_exitosos_porcentaje, '1_3_centros_exitosos_porcentaje.png')

#1.4 Acciones último tercio
hvplot.save(acciones_profundidad, '1_4_acciones_profundidad.png') # pases y centros profundos
hvplot.save(entradas_area, '1_4_entradas_area.png')
hvplot.save(ratio_eficacia, '1_4_ratio_eficacia.png') #entradas al área / goles
hvplot.save(preferencia_entradas_area, '1_4_preferencia_entradas_area.png') #forma de entrar al área, en conducción o centro
hvplot.save(toques_dentro_área, '1_4_toques_dentro_área.png') #toques de balón dentro del área

#1.5 Duelos aéreos
hvplot.save(duelos_areos, '1_5_duelos_areos.png') #Duelos aéreos / duelos aéreos ganados